In [1]:
import json
import math
import re
import time
from datetime import datetime
from string import punctuation, whitespace
from IPython.display import clear_output
import emoji
import numpy as np
import pandas as pd
from natasha import (PER, Doc, MorphVocab, NamesExtractor, NewsEmbedding,
                     NewsMorphTagger, NewsNERTagger, NewsSyntaxParser,
                     Segmenter)
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from dostoevsky.tokenization import RegexTokenizer
from dostoevsky.models import FastTextSocialNetworkModel
import pickle

In [2]:
df_raw = pd.read_csv("./reviews_cleaned.csv")
df_raw

,Unnamed: 0,review_full_text,review_rating,product,category,url,review_full_text_marked
0,0,Взяла эти ящики для рассады и не пожалела. Пл...,5.0,Мастерская Elbrus / Балконные ящики для цветов...,"Сад и дача / Горшки, опоры и все для рассады /...",https://www.wildberries.ru/catalog/144230143/f...,Unmarked
1,1,"Отличные ящики, к качеству и упаковке претензи...",5.0,Мастерская Elbrus / Балконные ящики для цветов...,"Сад и дача / Горшки, опоры и все для рассады /...",https://www.wildberries.ru/catalog/144230143/f...,Unmarked
2,2,"Быстрая доставка , все цело. И подарок положил...",5.0,Мастерская Elbrus / Балконные ящики для цветов...,"Сад и дача / Горшки, опоры и все для рассады /...",https://www.wildberries.ru/catalog/144230143/f...,Unmarked
3,3,"Ящики отличные, крепкие, маленький презент вну...",5.0,Мастерская Elbrus / Балконные ящики для цветов...,"Сад и дача / Горшки, опоры и все для рассады /...",https://www.wildberries.ru/catalog/144230143/f...,Unmarked
4,4,"Очень хорошие ящики, уже с отверстиями для вод...",5.0,Мастерская Elbrus / Балконные ящики для цветов...,"Сад и дача / Горшки, опоры и все для рассады /...",https://www.wildberries.ru/catalog/144230143/f...,Unmarked
...,...,...,...,...,...,...,...
3283,3283,Ужасная лампа. Греет очень сильно. Надолго вкл...,1.0,HOUSEFLOW / Фитолампа для растений и рассады п...,"Сад и дача / Горшки, опоры и все для рассады /...",https://www.wildberries.ru/catalog/166226613/f...,Unmarked
3284,3284,"Лампа в принципе не плохая,но с ДНаТ сложно ко...",3.0,HOUSEFLOW / Фитолампа для растений и рассады п...,"Сад и дача / Горшки, опоры и все для рассады /...",https://www.wildberries.ru/catalog/166226613/f...,Unmarked
3285,3285,"Очень сильно греется, нагревается буквально за...",3.0,HOUSEFLOW / Фитолампа для растений и рассады п...,"Сад и дача / Горшки, опоры и все для рассады /...",https://www.wildberries.ru/catalog/166226613/f...,Unmarked
3286,3286,"заказывал на 200вт. измерил, показывала 80 вт,...",2.0,HOUSEFLOW / Фитолампа для растений и рассады п...,"Сад и дача / Горшки, опоры и все для рассады /...",https://www.wildberries.ru/catalog/166226613/f...,Unmarked


In [3]:
stopwords_ru = stopwords.words("russian")
segmenter = Segmenter()
morph_vocab = MorphVocab()
emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)
ner_tagger = NewsNERTagger(emb)

In [77]:
def get_syntax(df, row, text_cell = "text"):
    res = []
    
    def recursive(df, row, res, text_cell):
        edges = df[(df["head_id"]==row["id"])]
        edges = edges[(edges["id"] != edges["head_id"])]
        if edges.empty:
            if re.search(fr"[{punctuation}]", row[text_cell]):
                return
            res.append((row[text_cell], row["rel"], row["pos"], row["lemma"]))
            return
        
        isPrinted = False
        for index, edge in edges.iterrows():
            if edge["rel"] == "conj":
                continue
            if text_cell == "lemma":
                if edge["text"] in stopwords_ru or re.search(fr"\d|[{punctuation}]", edge["text"]):
                    continue
            if int(edge["id"].split("_")[1]) > int(row["id"].split("_")[1]) and not isPrinted:
                res.append((row[text_cell], row["rel"], row["pos"], row["lemma"]))
                isPrinted = True
            recursive(df, edge, res, text_cell)
        if not isPrinted:
            res.append((row[text_cell], row["rel"], row["pos"], row["lemma"]))
            isPrinted = True
    
    recursive(df,row, res, text_cell)
    texts = [x[0] for x in res]
    texts = re.sub(r'\s([?.!,;:"](?:\s|$))', r'\1', " ".join(texts))
    rels = [x[1] for x in res]
    poses = [x[2] for x in res]
    lemmas = [x[3] for x in res]
    return texts, rels, poses, lemmas

In [5]:
numpy_data = df_raw.to_numpy()

In [6]:
black_list = ["спасибо", "доставка", "продавец", "покупка",
              "хороший", "отличный", "очень",
              "прийти"] + stopwords_ru

In [7]:
def isPos(data, pos, black_list = black_list):
    indices = [i for i, x in enumerate(data.iloc[0]) if x == pos]
    nouns = [data.iloc[1][i] for i in indices if data.iloc[1][i] not in black_list]
    return bool(nouns)

In [8]:
start = time.time()
[x for x in range(1222222)]
end = time.time()
print(end - start)
start = time.time()
[x for x in range(12222222)]
end = time.time()
print(end - start)

0.059784889221191406
0.5962908267974854


In [9]:
pd.set_option('display.max_rows', 40)

In [79]:
df_res = pd.DataFrame(columns=["text", "full_text", "rels", "poses", "lemmas"])
print_index = 1
for df_item in numpy_data:
    # clear_output(wait=True)
    res = df_item[df_raw.columns.get_indexer(["review_full_text"])[0]]
    doc = Doc(res)  # Doc(res.text)
    doc.segment(segmenter)
    doc.tag_morph(morph_tagger)
    doc.parse_syntax(syntax_parser)
    doc.tag_morph(morph_tagger)
    for token in doc.tokens:
        token.lemmatize(morph_vocab)
    data = doc.tokens
    columns = list(doc.tokens[0].as_json.keys())
    df_natasha = pd.DataFrame(data=data, columns=columns)
    df_objects = df_natasha[
        (df_natasha["rel"] == "root")
        | (df_natasha["rel"] == "conj")
        | (df_natasha["id"] == df_natasha["head_id"])
    ]
    temp = pd.DataFrame(
        df_objects.apply(lambda row: get_syntax(df_natasha, row), axis=1)
    )
    if temp.empty:
        continue
    temp[["text", "rels", "poses", "lemmas"]] = pd.DataFrame(
        list(temp[temp.columns[0]]), index=temp.index
    )
    temp = temp.drop([temp.columns[0]], axis=1)
    # result = temp[(temp["poses"].apply(lambda x: "NOUN" in x)) ]#& (temp["poses"].apply(lambda x: "ADJ" in x))]
    result = temp[
        (temp[["poses", "lemmas"]].apply(lambda x: isPos(x, "NOUN"), axis=1))
        & (temp[["poses", "lemmas"]].apply(lambda x: isPos(x, "ADJ"), axis=1))
    ]
    with pd.option_context('mode.chained_assignment', None):
        result["full_text"] = res
    df_res = pd.concat([df_res, result])

    display(fr"{ print_index } / { numpy_data.shape[0] }", clear=True)
    print_index += 1
df_res

'373 / 3288'

KeyboardInterrupt: 

In [11]:
df = df_res.reset_index()
df = df.drop(["index"], axis=1)
df

,text,full_text,rels,poses,lemmas
0,Пластик прочный,Взяла эти ящики для рассады и не пожалела. Пл...,"[nsubj, root]","[NOUN, ADJ]","[пластик, прочный]"
1,есть съёмный поддон,Взяла эти ящики для рассады и не пожалела. Пл...,"[root, amod, nsubj]","[VERB, ADJ, NOUN]","[быть, съемный, поддон]"
2,идеальные размеры,Взяла эти ящики для рассады и не пожалела. Пл...,"[amod, conj]","[ADJ, NOUN]","[идеальный, размер]"
3,красивый внешний вид,Взяла эти ящики для рассады и не пожалела. Пл...,"[amod, amod, conj]","[ADJ, ADJ, NOUN]","[красивый, внешний, вид]"
4,Отдельное спасибо продавцу за подарок,Взяла эти ящики для рассады и не пожалела. Пл...,"[amod, root, iobj, case, nmod]","[ADJ, NOUN, NOUN, ADP, NOUN]","[отдельный, спасибо, продавец, за, подарок]"
...,...,...,...,...,...
4214,Лампа принципе не плохая десять дней,"Лампа в принципе не плохая,но с ДНаТ сложно ко...","[nsubj, obl, advmod, root, nummod:gov, iobj]","[NOUN, NOUN, PART, ADJ, NUM, NOUN]","[лампа, принцип, не, плохой, десять, день]"
4215,но сложно в с ДНаТ конкурировать,"Лампа в принципе не плохая,но с ДНаТ сложно ко...","[cc, conj, case, case, obl, csubj]","[CCONJ, ADJ, ADP, ADP, NOUN, VERB]","[но, сложный, в, с, днат, конкурировать]"
4216,так и должно быть или это брак,"Очень сильно греется, нагревается буквально за...","[cc, fixed, conj, xcomp, cc, nsubj, nsubj]","[ADV, PART, ADJ, VERB, CCONJ, PRON, NOUN]","[так, и, должный, быть, или, это, брак]"
4217,измерил на следующей день,"заказывал на 200вт. измерил, показывала 80 вт,...","[root, case, amod, obl]","[VERB, ADP, ADJ, NOUN]","[измерить, на, следующий, день]"


In [12]:
df.reindex(df.text.str.len().sort_values(ascending=True).index)[2000:2100]

,text,full_text,rels,poses,lemmas
2420,и без повреждений за быструю поставку,"Отличный, качественный флорариум! Упаковано вс...","[advmod, case, conj, case, amod, nmod]","[CCONJ, ADP, NOUN, ADP, ADJ, NOUN]","[и, без, повреждение, за, быстрый, поставка]"
1147,Полный комплект все хорошего качества,Отличный парник! Очень рекомендую! Лёгок в сбо...,"[amod, nsubj, nsubj, amod, conj]","[ADJ, NOUN, PRON, ADJ, NOUN]","[полный, комплект, весь, хороший, качество]"
2191,Очень красивое кашпо хорошего размера,"Очень красивое кашпо хорошего размера, то что ...","[advmod, amod, root, amod, nmod]","[ADV, ADJ, NOUN, ADJ, NOUN]","[очень, красивый, кашпо, хороший, размер]"
1158,Стойка хорошая все детали в комплекте,"Стойка хорошая, все детали в комплекте. Собрал...","[nsubj, root, det, nsubj, case, nmod]","[ADJ, ADJ, DET, NOUN, ADP, NOUN]","[стойка, хороший, весь, деталь, в, комплект]"
2715,Родставку купила белую в подарок маме,Большое спасибо за качественное и красивое изд...,"[nsubj, root, obj, case, nmod, iobj]","[PROPN, VERB, ADJ, ADP, NOUN, NOUN]","[родставка, купить, белый, в, подарок, мама]"
...,...,...,...,...,...
2635,Доставка быстрая целая без повреждений,"Доставка быстрая, упаковка целая без поврежден...","[nsubj, amod, conj, case, nmod]","[NOUN, ADJ, ADJ, ADP, NOUN]","[доставка, быстрый, целый, без, повреждение]"
2917,Отличная подставка я думала будет хуже,"Отличная подставка, я думала будет хуже. Крепк...","[amod, nsubj, nsubj, advcl, cop, advcl]","[ADJ, NOUN, PRON, VERB, AUX, ADJ]","[отличный, подставка, я, думать, быть, хуже]"
1292,самый верхний это почти уже косяк вб 😣,"Пришли почти целые , самый верхний отколот нем...","[amod, amod, nsubj, advmod, advmod, amod, root...","[ADJ, ADJ, PRON, ADV, ADV, NOUN, NOUN, PUNCT]","[самый, верхний, это, почти, уже, косяк, вб, 😣]"
105,яркие пятна видимо но вид это не портит,"Хорошие ящики. Обычные. Упакованы в коробку, ц...","[amod, nsubj, advmod, cc, nsubj, nsubj, advmod...","[ADJ, NOUN, ADV, CCONJ, NOUN, PRON, PART, VERB]","[яркий, пятно, видимо, но, вид, это, не, портить]"


In [13]:
df["raw"] = [list(zip(*l)) for l in list(zip(df.lemmas, df.poses, df.rels))]

In [14]:
tokenizer = RegexTokenizer()
dostoevsky_model = FastTextSocialNetworkModel(tokenizer=tokenizer)
def foo(df, colname):
    messages = df[colname]
    results = dostoevsky_model.predict(messages, k=5)
    df[f"sentiment_{colname}"] = [{f"{k}_{colname}": round(v, 2) for k, v in dct.items()} for dct in results]
    df = pd.concat([df, df[f"sentiment_{colname}"].apply(pd.Series)], axis=1)
    df = df.drop([f"sentiment_{colname}"], axis=1)
    return df
df = foo(df, "text")
df = foo(df, "full_text")
df

C:\Users\Kirill\AppData\Local\Temp\ipykernel_3184\3312146909.py:7: FutureWarning: Returning a DataFrame from Series.apply when the supplied function returns a Series is deprecated and will be removed in a future version.
  df = pd.concat([df, df[f"sentiment_{colname}"].apply(pd.Series)], axis=1)
C:\Users\Kirill\AppData\Local\Temp\ipykernel_3184\3312146909.py:7: FutureWarning: Returning a DataFrame from Series.apply when the supplied function returns a Series is deprecated and will be removed in a future version.
  df = pd.concat([df, df[f"sentiment_{colname}"].apply(pd.Series)], axis=1)


,text,full_text,rels,poses,lemmas,raw,skip_text,neutral_text,positive_text,negative_text,speech_text,positive_full_text,neutral_full_text,skip_full_text,negative_full_text,speech_full_text
0,Пластик прочный,Взяла эти ящики для рассады и не пожалела. Пл...,"[nsubj, root]","[NOUN, ADJ]","[пластик, прочный]","[(пластик, NOUN, nsubj), (прочный, ADJ, root)]",0.09,0.09,0.02,0.02,0.00,0.21,0.14,0.10,0.08,0.04
1,есть съёмный поддон,Взяла эти ящики для рассады и не пожалела. Пл...,"[root, amod, nsubj]","[VERB, ADJ, NOUN]","[быть, съемный, поддон]","[(быть, VERB, root), (съемный, ADJ, amod), (по...",0.03,0.89,0.01,0.01,0.00,0.21,0.14,0.10,0.08,0.04
2,идеальные размеры,Взяла эти ящики для рассады и не пожалела. Пл...,"[amod, conj]","[ADJ, NOUN]","[идеальный, размер]","[(идеальный, ADJ, amod), (размер, NOUN, conj)]",0.00,0.99,0.01,0.09,0.00,0.21,0.14,0.10,0.08,0.04
3,красивый внешний вид,Взяла эти ящики для рассады и не пожалела. Пл...,"[amod, amod, conj]","[ADJ, ADJ, NOUN]","[красивый, внешний, вид]","[(красивый, ADJ, amod), (внешний, ADJ, amod), ...",0.07,0.73,0.31,0.00,0.00,0.21,0.14,0.10,0.08,0.04
4,Отдельное спасибо продавцу за подарок,Взяла эти ящики для рассады и не пожалела. Пл...,"[amod, root, iobj, case, nmod]","[ADJ, NOUN, NOUN, ADP, NOUN]","[отдельный, спасибо, продавец, за, подарок]","[(отдельный, ADJ, amod), (спасибо, NOUN, root)...",0.01,0.13,0.01,0.00,0.73,0.21,0.14,0.10,0.08,0.04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4214,Лампа принципе не плохая десять дней,"Лампа в принципе не плохая,но с ДНаТ сложно ко...","[nsubj, obl, advmod, root, nummod:gov, iobj]","[NOUN, NOUN, PART, ADJ, NUM, NOUN]","[лампа, принцип, не, плохой, десять, день]","[(лампа, NOUN, nsubj), (принцип, NOUN, obl), (...",0.14,0.09,0.12,0.18,0.00,0.07,0.26,0.28,0.32,0.01
4215,но сложно в с ДНаТ конкурировать,"Лампа в принципе не плохая,но с ДНаТ сложно ко...","[cc, conj, case, case, obl, csubj]","[CCONJ, ADJ, ADP, ADP, NOUN, VERB]","[но, сложный, в, с, днат, конкурировать]","[(но, CCONJ, cc), (сложный, ADJ, conj), (в, AD...",0.02,0.85,0.00,0.08,0.00,0.07,0.26,0.28,0.32,0.01
4216,так и должно быть или это брак,"Очень сильно греется, нагревается буквально за...","[cc, fixed, conj, xcomp, cc, nsubj, nsubj]","[ADV, PART, ADJ, VERB, CCONJ, PRON, NOUN]","[так, и, должный, быть, или, это, брак]","[(так, ADV, cc), (и, PART, fixed), (должный, A...",0.00,0.98,0.03,0.06,0.00,0.07,0.58,0.07,0.17,0.00
4217,измерил на следующей день,"заказывал на 200вт. измерил, показывала 80 вт,...","[root, case, amod, obl]","[VERB, ADP, ADJ, NOUN]","[измерить, на, следующий, день]","[(измерить, VERB, root), (на, ADP, case), (сле...",0.01,0.97,0.03,0.07,0.00,0.10,0.82,0.07,0.09,0.00


In [16]:
df.to_csv("./data/for_clustering.csv")

In [69]:
df_sub = df[(df["negative_full_text"] > 0.4) & (df["positive_full_text"] < 0.2)]
len(df_sub)

169

In [70]:
nouns = pd.Series(
    [
        item[0]
        for sublist in df_sub["raw"]
        for item in sublist
        if not (item[0] in stopwords_ru or re.search(rf"\d|[{punctuation}]", item[0]))
        and item[1] == "NOUN"
    ]
).value_counts()
nouns

горшок        11
ящик          10
упаковка      10
товар          9
качество       5
              ..
балкон         1
подарок        1
щедрость       1
склад          1
чередовать     1
Name: count, Length: 204, dtype: int64

In [71]:
df_nouns = df_sub
df_nouns["noun"] = df_nouns["raw"].apply(lambda x: 
    [
        item[0]
        for item in x
        if not (item[0] in stopwords_ru or re.search(rf"\d|[{punctuation}]", item[0]))
        and item[1] == "NOUN"
    ])
df_nouns = df_nouns.explode("noun")

df_x = df_nouns.groupby(["noun"])["text"].agg({
    ". ".join,
})
df_x.columns = [col[0] for col in df_x.columns]
# df_x[df_x.columns[0]].agg(len).sort_values(ascending=False)
df_nouns = pd.DataFrame(df_x[df_x.columns[0]])
df_nouns["len"] = df_nouns["j"].agg(len)
df_nouns = df_nouns.sort_values("len", ascending=False)
df_nouns

C:\Users\Kirill\AppData\Local\Temp\ipykernel_3184\3080119999.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_nouns["noun"] = df_nouns["raw"].apply(lambda x:
C:\Users\Kirill\AppData\Local\Temp\ipykernel_3184\3080119999.py:17: FutureWarning: using <built-in function len> in Series.agg cannot aggregate and has been deprecated. Use Series.transform to keep behavior unchanged.
  df_nouns["len"] = df_nouns["j"].agg(len)


,j,len
noun,,
горшок,Сам горшок не плохой. Не тот цветочный горшок. а пришли снаружи чёрные в белом кашпо коричневые быть в Нутри как дор горшок. Горшки не плохие для рассады. Хорошие горшки не особо толстый но гибкий при аккуратном обращении хватит больше чем на сезон. Нормальные транспортировочные горшки. Но почему сломанному по одному горшку отправляют. невозможно Эти горшки закрыть когда насыпана земля. Я полч...,547
товар,Сам товар был бы хорош если целый. я понимаю когда входе транспортировки ломается товар отломанные куски как это назвать что его таким Если положили находились бы. что везёте на пункт выдачи плохой товар. Товар не дорогой. Отвратительный товар ненадлежащего качества пришел разбитым. Заказала 2 шт одновременно Очень достойного качества Мало того что отправили разными посылками В остальном прете...,544
упаковка,я не могу понять только одного в без дополнительной упаковки пупырчатой ленты отправлять пластмассовые изделия обычном целофане. очень обидно почему нет никакой упаковки. плохая упаковка. Плохая упаковка. Оригинальная упаковка лечуза вскрыта. пришел с треснутым стеклом когда уже выкинула все упаковки. Упаковка колышки торчали в разные стороны. Рваная упаковка. Упаковка просто ужасна Жесть. Упа...,517
доставка,в дне там не трещина была все доставка так как принимала у курьера скорее не я😢 капец я понимаю через курьера сейчас напишете что вы что это виновата с потёртостями доставка Wildberries как можно. в дне там не трещина была все доставка так как принимала у курьера скорее не я😢 капец я понимаю через курьера сейчас напишете что вы что это виновата с потёртостями доставка Wildberries как можно. До...,442
ящик,Ящики неплохие. Упаковано хорошо детали отколотой небыло значит на складе заведомо кладут разбитые ящики. но упакованы Ящики то хорошие просто в полиэтиленовую пленку из за чего верхний пришол колотый. Замечательные ящики. Один ящик оказался с отбитым краем. Заказала два белых ящика. В целый ящик запихнули разбитый. Ящики глубокие. Ящики вроде не плохие. Итог возврат платный ввязаться не охота...,427
...,...,...
грызун,и погрыз грызун,15
опора,Удобная опора,13
мятый,Коробка мятая,13


In [72]:
def get_counts(corpus):
    try:
        regex = re.compile(r"(\sне)\s", re.IGNORECASE)
        corpus = regex.sub(r"\g<1>", corpus)
        
        doc_corpus = Doc(corpus)
        doc_corpus.segment(segmenter)
        doc_corpus.tag_morph(morph_tagger)
        doc_corpus.parse_syntax(syntax_parser)
        doc_corpus.tag_morph(morph_tagger)
        for token in doc_corpus.tokens:
            token.lemmatize(morph_vocab)
        data_natasha = doc_corpus.tokens
        columns = list(doc_corpus.tokens[0].as_json.keys())
        df_natasha = pd.DataFrame(data=data_natasha, columns=columns)
        
        df_adj = df_natasha[df_natasha["pos"] == "ADJ"]
        if len(df_adj.index) == 0:
            return []
        
        cv = CountVectorizer()   
        cv_fit = cv.fit_transform(df_adj['lemma'])    
        word_list = cv.get_feature_names_out() 
        count_list = cv_fit.toarray().sum(axis=0)
        df_counts = pd.DataFrame.from_dict(zip(word_list,count_list))
        df_counts = df_counts.rename({0: 'token', 1: 'count'}, axis=1)
        df_counts = df_counts.sort_values(["count"], ascending=False)
        df_counts = df_counts[df_counts["count"] > 1]
        
        return df_counts.to_dict('tight')["data"]
    except:
        return []

In [73]:
pd.set_option('display.max_rows', 20)

In [74]:
noun_lemmas = df_nouns["j"].agg(get_counts)
df_noun_words_counts = pd.DataFrame(noun_lemmas)
df_noun_words_counts

C:\Users\Kirill\AppData\Local\Temp\ipykernel_3184\2695223464.py:1: FutureWarning: using <function get_counts at 0x0000019C8F9ED670> in Series.agg cannot aggregate and has been deprecated. Use Series.transform to keep behavior unchanged.
  noun_lemmas = df_nouns["j"].agg(get_counts)


,j
noun,
горшок,"[[неплохой, 2]]"
товар,[]
упаковка,"[[плохой, 2]]"
доставка,"[[виноватый, 2], [нея, 2], [потертость, 2]]"
ящик,"[[неплохой, 2]]"
...,...
грызун,[]
опора,[]
мятый,[]


In [75]:
pd.set_option('max_colwidth', 400)
pd.set_option('display.max_rows', 25)

In [76]:
df_noun_words_counts[df_noun_words_counts["j"].agg(len) > 0]

C:\Users\Kirill\AppData\Local\Temp\ipykernel_3184\3500821967.py:1: FutureWarning: using <built-in function len> in Series.agg cannot aggregate and has been deprecated. Use Series.transform to keep behavior unchanged.
  df_noun_words_counts[df_noun_words_counts["j"].agg(len) > 0]


,j
noun,
горшок,"[[неплохой, 2]]"
упаковка,"[[плохой, 2]]"
доставка,"[[виноватый, 2], [нея, 2], [потертость, 2]]"
ящик,"[[неплохой, 2]]"
курьер,"[[виноватый, 2], [нея, 2], [потертость, 2]]"
черный,"[[черный, 2]]"
лист,"[[недержать, 2], [пластиковый, 2]]"
полка,"[[невставить, 2], [разный, 2]]"
интерьер,"[[черный, 2]]"
